This notebook is a sample code with Japanese comments.

# 2.1 まずはsubmit！　順位表に載ってみよう

パッケージの読み込み

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling

## データの読み込み

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
gender_submission.head()

In [ ]:
gender_submission.shape

In [ ]:
data = pd.concat([train, test], sort=False)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
print(len(train), len(test), len(data))

In [ ]:
data.isnull().sum()

## 特徴量エンジニアリング

### 2. Sex

In [ ]:
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)

In [ ]:
data.head()

### 3. Embarked

In [ ]:
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

### 4. Fare

In [ ]:
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

### 5. Age

In [ ]:
data["Age"].fillna(data["Age"].median(),inplace=True)

### 6.FamilySize(SibSp+Parch) 

In [ ]:
import seaborn as sns

data["FamilySize"]=data["SibSp"]+data["Parch"]+1
train["FamilySize"]=data["FamilySize"][:len(train)]


sns.countplot(x="FamilySize", data=train, hue="Survived")

### 7.IsAone 

In [ ]:
data["IsAlone"]=0
data.loc[data["FamilySize"]==1,"IsAlone"]=1

### 

##  新しいデータフレームの作成 

In [ ]:
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

In [ ]:
train = data[:len(train)]
test = data[len(train):]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
y_train = train['Survived']
x_train = train.drop('Survived', axis=1)
x_test = test.drop('Survived', axis=1)

In [ ]:
x_train.head()

In [ ]:
x_train.shape

In [ ]:
y_train.head()

In [ ]:
x_test.head()

## 機械学習アルゴリズム

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.3,random_state=0)

In [ ]:
categorical_features=["Embarked","Pclass","Sex"]

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_train=lgb.Dataset(x_train,y_train,categorical_feature=categorical_features)

In [ ]:
lgb_val=lgb.Dataset(x_val,y_val,categorical_feature=categorical_features)

In [ ]:
params={"objective":"binary"}

In [ ]:
model=lgb.train(
    params,lgb_train,
    valid_sets=[lgb_train,lgb_val],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

In [ ]:
y_pred=model.predict(x_test,num_iteretion=model.best_ieteration)

In [ ]:
y_pred

## 提出

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('titanic.csv', index=False)